# <b><span style='color:#38c9ff;font-size:150%'>Sensorización y Machine Learning</span><br>Por: Ruben Durango  </b>
---
## 📃| **Introducción**
Pendiente

## 🔢| **Objetivos**



## <b><span style='color:#38c9ff'>0</span> | Inicializacion del Notebook</b>

In [2]:
# Inicializacion del Notebook

import numpy as np # Libreria para trabajar con matrices
import pandas as pd # Libreria para trabajar con dataframes
import scipy.stats # Libreria para trabajar con estadisticas

import matplotlib.pyplot as plt # Libreria para trabajar con graficas
from matplotlib.ticker import # Libreria para trabajar con graficas
from cycler import cycler # Libreria para trabajar con graficas
from IPython.display import display # Libreria para presentar informacion

# Librerias para modelado
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import GroupKFold, cross_val_score
from sklearn.ensemble import HistGradientBoostingRegressor, HistGradientBoostingClassifier
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import roc_auc_score, roc_curve
from xgboost import XGBClassifier

# Valores base para matplotlib
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True, figsize=(11, 5))
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
    facecolor = '#f0f3f4',
)
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
    legend=False,
)

%config InlineBackend.figure_format = 'retina'

# <div style="color:white;display:fill;border-radius:5px;background-color:#38c9ff;letter-spacing:0.5px;overflow:hidden"><p style="padding:20px;color:#FFFFFF;overflow:hidden;margin:0;font-size:110%"><b>1 |</b>  Data Overview</p></div>

## <b><span style='color:#38c9ff'>1.1</span> | Main Goal</b>
In this competition, you'll classify 60-second sequences of sensor data, indicating whether a subject was in either of two activity states for the duration of the sequence.

## <b><span style='color:#38c9ff'>1.2</span> | Files and Field Descriptions</b>

1. **train.csv** - the training set, comprising ~26,000 60-second recordings of thirteen biological sensors for almost one thousand experimental participants
    - `sequence` - a unique id for each sequence
    - `subject` - a unique id for the subject in the experiment
    - `step` - time step of the recording, in one second intervals
    - `sensor_00 - sensor_12` - the value for each of the thirteen sensors at that time step
2. **train_labels.csv** - the class label for each sequence.
    - `sequence` - the unique id for each sequence.
    - `state` - the state associated to each sequence. This is the target which you are trying to predict.
3. **test.csv** - the test set. For each of the ~12,000 sequences, you should predict a value for that sequence's state.